# Simple Convolutional Neural Network in Keras

Keras was developed to make it easier for deep learning engineers to prototype many different models very quickly.  It is a higher level framework than not only Python, but TensorFlow, too.  As such it comes with both the benefits (simplicity and ease of use) and the disadvantages (restrictive) that additional abstractions provide.

## Smile 

This notebook contains the code to create a convolutional neural network that can determine if a person in an image is smiling or not.  The data being used is contained within the test_happy.h5 and train_happy.h5 files; these need to be placed in a folder named 'datasets' and then that folder needs to be placed in the same location as this notebook file.

## Import Modules

In [ ]:
import numpy as np
import h5py
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

## Helper Functions

In [ ]:
def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def load_dataset():
    train_dataset = h5py.File('datasets/train_happy.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # train set labels

    test_dataset = h5py.File('datasets/test_happy.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # test set labels

    classes = np.array(test_dataset["list_classes"][:]) # list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

## Prepare Dataset

The cell below loads and normalizes the dataset and prints out information about its shapes.

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

##  Create the Model


Keras uses a different convention with variable names than used with NumPy and TensorFlow. In particular, rather than creating and assigning a new variable on each step of forward propagation such as `X`, `Z1`, `A1`, `Z2`, `A2`, etc, for the computations for the different layers, in Keras code each line above just reassigns `X` to a new value using `X = ...`. In other words, during each step of forward propagation, the latest value in the commputation is written into the same variable `X`. The only exception is `X_input`, which is kept separate and is not overwritten, since it is  needed at the end to create the Keras model instance (`model = Model(inputs = X_input, ...)` above). 

In [ ]:
def HappyModel(input_shape):
    
    # Define the input placeholder
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (5, 5), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Instantiate model
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
    return model

    """
    Implementation of the HappyModel.
    
    Arguments:
    input_shape = shape of images

    Returns:
    model = a Model() instance in Keras
    """

## Train and Test the Model

To train and test this model, there are four steps in Keras:
1. Create the model by calling the function above
2. Compile the model by calling `model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])`
3. Train the model on train data by calling `model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)`
4. Test the model on test data by calling `model.evaluate(x = ..., y = ...)`

In [ ]:
#Create the model
happyModel = HappyModel(X_train.shape[1:])

In [ ]:
# Compile the model
happyModel.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
# Train the model
happyModel.fit(x = X_train, y = Y_train, epochs = 2, batch_size = 16)

In [ ]:
# Test/evaluate the model
preds = happyModel.evaluate(x = X_test, y = Y_test)
### END CODE HERE ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

A useful feature of Keras is `model.summary()`, which prints the details of the layers of the model in a table with the sizes of its inputs/outputs

In [ ]:
happyModel.summary()